In [1]:
import numpy as np
import scipy as sc
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import ndcg_score
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

# В этом ноутбуке обучаем (хотя бы пытаемся) нейронку для рекоммендациий

Идея такая же как в ALS - делаем матричное разложение. Однако тут понадобятся и отрицательные примеры, так что в target оставляем и 0 и 1. Модель базовой нейронки: 2 эмбеддинга (для пользователей и песен) + 2 линейных для их отображения. После берем их скалярное произведение, используем сигмоиду и получаем вес для рекомендации. Как лосс используем Binary Cross Entropy. После обучения нейронки, прогоняем данные всех пользователей и песен и получаем 2 матрицы: users_embs = (n_users x n_factors) и songs_embs = (n_factors x n_songs). Матрицу рекомендаций получаем как произведение этих двух матриц (аналогично с ALS).

Также тут мы можем использовать данные пользователей и песен, использовав эмбеддинги на их фичи. После соеденяем эмбединги юзера с эмбедингами его фичей и используем эти данные для предсказания.

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epochs = 10
batch_sz = 128

In [3]:
# Read the data
train = pd.read_csv("train_processed.csv", index_col=0)
val = pd.read_csv("val_processed.csv", index_col=0)
test = pd.read_csv("test_processed.csv", index_col=0)

In [4]:
train = train[(train["song_id"] >= 0) & (train["msno"] >= 0)]
val = val[(val["song_id"] >= 0) & (val["msno"] >= 0)]
test = test[(test["song_id"] >= 0) & (test["msno"] >= 0)]

In [5]:
# We can't use "source_system_tab", "source_screen_name", "source_type" on evaluation stage, then we drop it
train = train.drop(["source_system_tab", "source_screen_name", "source_type"], axis=1)
val = val.drop(["source_system_tab", "source_screen_name", "source_type"], axis=1)
test = test.drop(["source_system_tab", "source_screen_name", "source_type"], axis=1)

In [6]:
members_data = pd.read_csv("members_data_processed.csv", index_col=0)
songs_data = pd.read_csv("songs_data_processed.csv", index_col=0)

In [7]:
members_data

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,0,1,0,undefiend,7,20110820,20170920
1,1,1,0,undefiend,7,20150628,20170622
2,2,1,0,undefiend,4,20160411,20170712
3,3,1,0,undefiend,9,20150906,20150907
4,4,1,0,undefiend,4,20170126,20170613
...,...,...,...,...,...,...,...
34398,34398,1,0,undefiend,7,20131111,20170910
34399,34399,4,2,male,3,20141024,20170518
34400,34400,1,0,undefiend,7,20130802,20170908
34401,34401,1,0,undefiend,7,20151020,20170920


In [8]:
songs_data

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,0,247640,465,張信哲 jeff chang,董貞,何啟弘,3
1,1,197328,444,blackpink,teddy| future bounce| bekuh boom,teddy,31
2,2,231781,465,super junior,NaN,NaN,31
3,3,273554,465,s.h.e,湯小康,徐世珍,3
4,4,140329,726,貴族精選,traditional,traditional,52
...,...,...,...,...,...,...,...
2296315,2296315,20192,958,catherine collard,robert schumann 1810-1856,NaN,-1
2296316,2296316,273391,465,紀文惠 justine chi,NaN,NaN,3
2296317,2296317,445172,1609,various artists,NaN,NaN,52
2296318,2296318,172669,465,peter paul mary,NaN,NaN,52


In [9]:
# Бейзлайн для нейронки. Тут не используем данные о пользователях и песнях
# По сути та же факторизация матрицы, но через эмбеддинги
class BaseNN(nn.Module):
    def __init__(self, n_users, n_songs, n_factors=50, embedding_dropout=0.02, dropout_rate=0.2):
        super().__init__()
        self.n_factors = n_factors
        self.users = nn.Embedding(n_users, n_factors)
        self.songs = nn.Embedding(n_songs, n_factors)
        
        self.users_fc = nn.Linear(n_factors, n_factors)
        self.songs_fc = nn.Linear(n_factors, n_factors)
        
        self.relu = nn.ReLU()
        
        self._init()
    
    def forward(self, users, songs):
        u_emb = self.relu(self.users_fc(self.users(users)))
        s_emb = self.relu(self.songs_fc(self.songs(songs)))
        
        out = torch.sum(u_emb * s_emb, dim=1)
        return torch.sigmoid(out)
        
    
    def _init(self):
        """
        Initialize embeddings and hidden layers weights with xavier.
        """
        def init(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)

        self.users.weight.data.uniform_(-0.05, 0.05)
        self.songs.weight.data.uniform_(-0.05, 0.05)
        init(self.users_fc)
        init(self.songs_fc)
    
    def generate_user_and_song_matrices(self, user_list, song_list, batch_size=64, device='cpu'):
        out_u = torch.zeros((len(user_list), self.n_factors), device=device)
        
        for idx in range(0, len(user_list), batch_size):
            end_idx = min(idx + batch_size, len(user_list))
            u_emb = self.relu(self.users_fc(self.users(user_list[idx:end_idx])))
            out_u[idx:end_idx] = u_emb
        
        
        out_s = torch.zeros((len(song_list), self.n_factors), device=device)
        
        for idx in range(0, len(song_list), batch_size):
            end_idx = min(idx + batch_size, len(song_list))
            s_emb = self.relu(self.songs_fc(self.songs(song_list[idx:end_idx])))
            out_s[idx:end_idx] = s_emb
        
        return out_u, out_s
    

In [10]:
model = BaseNN(len(members_data), len(songs_data)).to(device)
loss_fn = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
n_epochs = 10
batch_size = 16

In [11]:
train_users = torch.tensor(list(train['msno']))
train_songs = torch.tensor(list(train['song_id']))
train_gt = torch.tensor(list(train['target']))


for epoch in range(n_epochs):
    total_loss = 0
    acc = 0
    for idx in tqdm(range(0, len(train_users), batch_size), total=len(train_users) // batch_size):
        end_idx = end_idx = min(idx + batch_size, len(train_users))

        inp_u = train_users[idx:end_idx].to(device)
        inp_s = train_songs[idx:end_idx].to(device)
        target = train_gt[idx:end_idx].to(device)

        optimizer.zero_grad()
        pred = model(inp_u, inp_s)
        loss = loss_fn(pred, target.float())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.detach().cpu().item() * (end_idx-idx)
        acc += torch.sum((pred > 0.5) == target).detach().cpu()
        
    total_loss /= len(train_users)
    print(f"Epoch {epoch}, loss {total_loss:.4f}")

  1%|▍                                                                        | 1593/276649 [03:52<11:09:54,  6.84it/s]


KeyboardInterrupt: 

## Нейронке надо 11 часов на обучение на 1 эпоху на GPU :skull:. К сожалению столько времени нет.